## Importowanie bibliotek
Biblioteka `morfeusz2` posłuży do lematyzacji, `plwordnet` i Słowosieć do analizy sentymentu. 

>**Słowosieć** (ang. PlWordNet) – baza danych leksykalno-semantycznych języka polskiego typu wordnet. Zawiera zestawy synonimicznych jednostek leksykalnych (synsety) opisanych krótkimi definicjami. Słowosieć służy jako słownik, w którym pojęcia (synsety) i poszczególne znaczenia wyrazów (jednostki leksykalne) zdefiniowane są poprzez miejsce w sieci wzajemnych relacji, odzwierciedlających system leksykalny polszczyzny. Słowosieć jest także wykorzystywana jako jeden z podstawowych zasobów do budowy programów przetwarzających język polski. ([Wikipedia](https://pl.wikipedia.org/wiki/S%C5%82owosie%C4%87))


- http://morfeusz.sgjp.pl/download/
- https://slowosiec.pl/

In [29]:
# import plwordnet
import morfeusz2
import pandas as pd

## Wstępne przetwarzanie tekstu
W tym punkcie podejmiemy następujące kroki:
1. Usunięcie niepotrzebnych znaków z tekstu źródłowego
2. Lematyzacja za pomocą Morfeusza
3. Usunięcie stop słów z lematów
   
Na początku załadujmy polski słownik sentymentu (ręcznie przefiltrowany `plwordnet` w csv). 

In [18]:
slownik_sentymentu = pd.read_csv('słowniki/SłownikSentymentu.csv', sep=';',encoding='cp1250')
# print(slownik_sentymentu.columns.tolist())

Wczytujemy zescrapowane pliki tekstowe z Reddita/Wykopu i dzielimy na pojedyncze słowa. Na samym końcu tworzymy ramkę danych, gdzie każde słowo jest w osobnym wierszu.

In [19]:
with open('data/results_reddit.txt', 'r', encoding='utf-8') as file:
    tekst_cały = file.read()
slowa = tekst_cały.split()
ramka_slow = pd.DataFrame(slowa, columns=['słowa'])

Kolejnym krokiem będzie przeczyszczenie wczytanego tekstu, czyli:
- usunięcie znaków interpunkcyjnych
- usunięcie niepożądanych słów z tekstu źródłowego
- usunięcie wszystkich linków z wpisów
- usunięcie liczb
- usunięcie polskich stop słów
  
W tym celu korzystamy z prostych list comprehensions.

In [21]:
# Znaki interpunkcyjne
to_remove = '-,? .>#:"/[]+=!%<*()|–„'
table = str.maketrans('', '', to_remove)
cleaned_slowa = [word.translate(table) for word in slowa]

# Niechciane słowa
remove_slowa = ['Tytuł:','tytuł:', 'Treść:','treść:','Tag:', 'Subreddit:']
filtered_words = [word for word in cleaned_slowa if word not in remove_slowa and word != '']
# Linki
filtered_words_no_links = [
    word for word in filtered_words
    if not (word.startswith('http') or word.startswith('www'))
]
# Liczby
filtered_words_no_links_no_digits = [
    word for word in filtered_words_no_links
    if not word.isdigit()
]

Na koniec tworzymy ramkę danych z przefiltrowanymi słowami.

In [22]:
ramka_slow = pd.DataFrame(filtered_words_no_links_no_digits, columns=['slowa'])

### Lematyzacja
O Morfeuszu twórcy piszą, że w skrócie to słownik morfologiczny.
Matura z polskiego już za nami za równo w wersji podstawowej, jak i rozszerzonej, a z Morfeusza korzystamy w wersji ze słownikiem gramatyki języka polskiego - [SGJP](http://sgjp.pl/o-slowniku/#liczby). 

**Ważne pojęcia** – czytamy dokumentację twórców
>Celem hasłowania (lematyzacji) jest wskazanie dla każdego słowa tekstowego opisującej je jednostki słownika morfologicznego (leksemu). Jest to więc analiza morfologiczna (lub tagowanie) ograniczona tylko do części informacji o formach — do lematów.

> Przybliżone hasłowanie polegające na odcięciu ze słów części zmieniającej się przy odmianie bywa nazywane stemowaniem. Metoda ta ma sens dla języków o ograniczonej fleksji, ale dla polskiego daje wyniki wysoce niezadowalające. W kontekście Morfeusza mówimy więc o prawdziwym hasłowaniu.

([Analizator morfologiczny Morfeusz](http://morfeusz.sgjp.pl/doc/about/)) 

In [27]:
morfeusz = morfeusz2.Morfeusz()
def lemmatize_word(word):
    analyses = morfeusz.analyse(word)
    if analyses:
        # analyses to lista krotek: (start, end, (lemma, tag), score)
        # Bierzemy pierwszą lematę z analizy
        lemma = analyses[0][2][1]  # [0] - pierwszy wynik, [2] - tuple (base, lemma, tag), [1] - lemma
        return lemma
    else:
        return word

ramka_slow['lemma'] = ramka_slow['slowa'].apply(lemmatize_word)

Najczęstsze słowa

In [28]:
ramka_slow['lemma_clean'] = ramka_slow['lemma'].apply(lambda x: x.split(':')[0])
lemma_counts = ramka_slow['lemma_clean'].value_counts()
# print(lemma_counts.head(50))

W końcu możemy przeczyścić lematy ze zbędnych słów - stopwords (się, na, i, w, itp...). W pliku `stopwords-pl.txt` znajdują się wszystkie stopsłowa. Pochodzi on z https://github.com/stopwords-iso/stopwords-pl

In [25]:
with open('słowniki/stopwords-pl.txt', 'r', encoding='utf-8') as file:
    df_polish_stop_words = file.read().splitlines()
ramka_slow = ramka_slow[~ramka_slow['lemma_clean'].isin(df_polish_stop_words)]
print(ramka_slow['lemma_clean'].value_counts().head(50))

lemma_clean
prezydent     182
treść         105
tytuł         104
wybory         86
kandydat       79
oda            60
państwo        58
partia         53
polski         47
rok            46
chcieć         43
głowa          43
tenże          42
osoba          40
móc            40
dzień          36
polityczny     35
swój           35
głos           34
urząd          34
zostać         34
władza         34
pierwszy       33
Polski         33
USA            32
lato           32
Polska         31
wyborczy       31
czas           30
głosować       30
kraj           30
mówić          29
maić           29
rząd           28
kampania       27
siebie         26
strona         26
brak           26
RP             26
spotkać        26
Trump          26
człowiek       26
stan           25
wiedzieć       25
ustawa         25
ukraina        25
drugi          25
wynik          24
mamić          24
wysoki         23
Name: count, dtype: int64


## Analiza sentymentu

In [26]:
ramka_slow = ramka_slow.merge(
    slownik_sentymentu,
    left_on='lemma_clean', 
    right_on='lemat',
    how='left'
)